In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
%matplotlib inline
from sklearn.model_selection import GridSearchCV
from matplotlib.pyplot import rcParams
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/titanic'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df=pd.read_csv("../input/titanic/train.csv")
train_df.head()

In [ ]:
test_df=pd.read_csv("../input/titanic/test.csv")
test_df.head()

In [ ]:
test_df.shape

In [ ]:
train_df.isnull().sum()

In [ ]:
test_df.isnull().sum()

In [ ]:
train_df=train_df.fillna(0)

In [ ]:
test_df=test_df.fillna(0)

In [ ]:
test_df.isna().sum()

In [ ]:
train_df.isna().sum()

In [ ]:
train_df=pd.get_dummies(train_df,columns=['Sex','Embarked'])
test_df=pd.get_dummies(test_df,columns=['Sex','Embarked'])

In [ ]:
train_df.columns

In [ ]:
test_df.columns

In [ ]:
num_col=train_df.select_dtypes(exclude="object").columns

In [ ]:
from scipy.stats import skew
skew_col=train_df[num_col].apply(lambda x: skew(x))
skew_col=skew_col[skew_col>0.5].index
train_df[skew_col]=np.log1p(train_df[skew_col])

In [ ]:
x=np.asanyarray(train_df[['Pclass',
        'Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q',
       'Embarked_S']])
x[:5]

In [ ]:
y=np.asanyarray(train_df.Survived)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,random_state=9)

In [ ]:
test_df['Embarked_S'].nunique()

In [ ]:
test_set=test_df[['Pclass','Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q',
       'Embarked_S']]

In [ ]:
leaf_size=list(range(1,10))
n_neighbors=list(range(1,10))
p=[1,2]
hp=dict(leaf_size=leaf_size,n_neighbors=n_neighbors,p=p)
knn=KNeighborsClassifier()
clf=GridSearchCV(knn,hp,cv=10)
best_model=clf.fit(x_train,y_train)

In [ ]:
best_model.best_estimator_.get_params()['leaf_size']

In [ ]:
best_model.best_estimator_.get_params()['n_neighbors']

In [ ]:
best_model.best_estimator_.get_params()['p']

In [ ]:
reg=KNeighborsClassifier(n_neighbors=6,p=1,leaf_size=1)
reg.fit(x,y)
y_pred=reg.predict(test_set)

In [ ]:
reg=XGBClassifier(max_depth=2,random_state=42)
reg.fit(x,y)
y_pred=reg.predict(test_set)

In [ ]:
test_df['Survived']=pd.Series(y_pred.reshape(1,-1)[0])
submission=pd.concat([test_df['PassengerId'],test_df['Survived']],axis=1)
submission.to_csv('submission.csv',index=False)